### Data prep

- Downloaded CSV of travis dataset 
- use first 100 commits of dataset 

- extract commits id & project name of those commits
- use values to get corresponding pom files (previous and current)
- use differ to extract information from pom files
- use this as data and the build result of the 100 travis commits as labels

# TODO

- take care of labels as they are currently gotten in a seperate step :D 

In [3]:
#!pip3 install pandas
import pandas as pd 

# TODO just use if original travistorrent dataset is present instead of smaller version 
# TODO think of duplicates in commitID
# size down original file
#df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_5_3_2016.csv')
#df = df.drop_duplicates()[0:100]
#print(df.head())
# - shuffle dataframe before generating subset to have more balance in labels 
#df = df.sample(frac = 1) # sample all rows without removing a fraction (100%)
#df.head()
# TODO save to data instead of datapre
#df.to_csv('travistorrent_100commits_withoutduplicates.csv', sep=',')
#df.to_csv('travistorrent_100commits_withoutduplicates_shuffeld.csv', sep=',')
# TODO add path os.join etc such that it is saved into data folder


In [4]:
# read in file 
df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_100commits_withoutduplicates_shuffeld.csv')
print(df.describe())

       Unnamed: 0         row  git_merged_with  gh_team_size  git_num_commits  \
count  100.000000  100.000000              0.0    100.000000       100.000000   
mean    49.500000   50.500000              NaN      3.720000         1.080000   
std     29.011492   29.011492              NaN      0.451261         0.393893   
min      0.000000    1.000000              NaN      3.000000         1.000000   
25%     24.750000   25.750000              NaN      3.000000         1.000000   
50%     49.500000   50.500000              NaN      4.000000         1.000000   
75%     74.250000   75.250000              NaN      4.000000         1.000000   
max     99.000000  100.000000              NaN      4.000000         3.000000   

       gh_num_issue_comments  gh_num_commit_comments  gh_num_pr_comments  \
count                  100.0                   100.0               100.0   
mean                     0.0                     0.0                 0.0   
std                      0.0              

In [7]:
df.head()

,Unnamed: 0,row,git_commit,gh_project_name,gh_is_pr,git_merged_with,gh_lang,git_branch,gh_first_commit_created_at,gh_team_size,...,tr_tests_skipped,tr_failed_tests,tr_testduration,tr_purebuildduration,tr_tests_ran,tr_tests_failed,git_num_committers,tr_num_jobs,tr_prev_build,tr_ci_latency
0,2,3,7086c22bb72fa58e07968a292b3c06adce0491c2,47deg/appsly-android-rest,False,NaN,java,master,2013-08-17 00:41:08,3,...,NaN,NaN,NaN,NaN,False,False,1,4,-1.0,-1
1,69,70,8c95d75168f2ea1cff49ae809c4baa6b2725bc8a,47deg/appsly-android-rest,False,NaN,java,master,2013-11-15 08:06:18,4,...,NaN,NaN,NaN,NaN,False,False,1,4,5122.0,-1
2,14,15,c08c4cdee730059ef0ddc0c55a6c5b6e7538af26,47deg/appsly-android-rest,False,NaN,java,master,2013-08-17 00:48:52,3,...,NaN,NaN,NaN,NaN,False,False,1,4,-1.0,-1
3,27,28,3ab53e42f0fb599832a0963a39ca94547239d89a,47deg/appsly-android-rest,False,NaN,java,master,2013-09-04 12:58:38,3,...,NaN,NaN,NaN,NaN,False,False,1,4,-1.0,-1
4,81,82,c67e65a39f0e87f546b9ece9c82a332b357fc034,47deg/appsly-android-rest,False,NaN,java,master,2013-12-06 12:01:02,4,...,NaN,NaN,NaN,NaN,False,False,1,4,3417.0,-1


#### Get DataFrame with information on project and commit of which we have build results

In [5]:
commit_information = pd.DataFrame(df)
commit_information = commit_information[['gh_project_name', 'git_commit', 'tr_status']]#.drop_duplicates()[0:100]
print(commit_information) # now we have a dataframe containing 
print('Size of dataframe (rows): ', commit_information.shape[0])

              gh_project_name                                git_commit  \
0   47deg/appsly-android-rest  7086c22bb72fa58e07968a292b3c06adce0491c2   
1   47deg/appsly-android-rest  8c95d75168f2ea1cff49ae809c4baa6b2725bc8a   
2   47deg/appsly-android-rest  c08c4cdee730059ef0ddc0c55a6c5b6e7538af26   
3   47deg/appsly-android-rest  3ab53e42f0fb599832a0963a39ca94547239d89a   
4   47deg/appsly-android-rest  c67e65a39f0e87f546b9ece9c82a332b357fc034   
..                        ...                                       ...   
95  47deg/appsly-android-rest  47e39ce40e3f03718e7ef9101701cccfe8610c33   
96  47deg/appsly-android-rest  f88e4b1eee613a8f6719bf274d44e23736779572   
97  47deg/appsly-android-rest  35a32f0135d094bfe752af1298b4c050491c295d   
98  47deg/appsly-android-rest  7086c22bb72fa58e07968a292b3c06adce0491c2   
99  47deg/appsly-android-rest  13ad8c06bef212b9270eea5547a109da87e4e866   

   tr_status  
0    errored  
1    errored  
2    errored  
3    errored  
4    errored  
..       

#### Clone project, checkout commit id and download current pom and pom before the commit 

In [9]:
# Imports
#!pip3 install GitPython
import git 
import shutil
import os

In [7]:
# path to pom file of a project using maven 
prefix_project = '/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/dataprep/'
path_to_pom = prefix_project+'projects/pom.xml'
prefix_url = 'https://github.com/'

# method for getting Pom file from commit id 
import shutil
def getPomFile(project, commit_id, path_to_pom, path_to_save_to):
    project.git.checkout(commit_id) # checkout commit of data to receive pom.xml file
    shutil.copy(path_to_pom, path_to_save_to)
    #!cp path_to_pom path_to_save_to

In [15]:
for index, row in commit_information.iterrows():
    # do it for every commit 
    commit_id = row['git_commit']
    project_name = row['gh_project_name']

    project_url = prefix_url+project_name
    if index > 0:
        shutil.rmtree('projects')
    project = git.Repo.clone_from(project_url, '/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/dataprep/projects')

    # TODO adapt naming of subfolders (commit id's are present multiple times so i stayed with indices for now)
    base_path = prefix_project+'pom_files/'+str(index)#+'/'+project_name+'/'+commit_id
    os.makedirs(base_path)

    path_to_save_to_current = base_path+'/pom_current.xml'
    path_to_save_to_previous = base_path+'/pom_previous.xml'

    getPomFile(project, commit_id, path_to_pom, path_to_save_to_current)
    prev_commit_id = commit_id+'^1'
    project.git.checkout(prev_commit_id) # receive pom.xml file before commit was done
    getPomFile(project, prev_commit_id, path_to_pom, path_to_save_to_previous)
    with open(base_path+"/build_result.csv", "w") as file:
        file.write(row['tr_status'])

    # TODO run for all 100 commits to get all pom files into working directory for differ :) 
    if index == 1:
        break
    print(commit_id)


7086c22bb72fa58e07968a292b3c06adce0491c2
